In [6]:
%cd "/content/drive/MyDrive/Grocery"

/content/drive/MyDrive/Grocery


### Data without augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pandas as pd
import numpy as np
import os


In [22]:
train_dir = os.path.join(os.getcwd(), 'train')
val_dir = os.path.join(os.getcwd(), 'test')

In [9]:
train_data = ImageDataGenerator(rescale=1/.255, 
                                preprocessing_function=preprocess_input)
val_data = ImageDataGenerator(rescale=1/.255, 
                                preprocessing_function=preprocess_input)

In [23]:
train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=10,
    seed=42,
)
val_generator = val_data.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=10,
    seed=42,
)

Found 120 images belonging to 30 classes.
Found 30 images belonging to 30 classes.


In [11]:
for data_batch, labels_batch in train_generator:
  print(data_batch.shape, labels_batch.shape)
  break

(10, 224, 224, 3) (10, 30)


In [12]:
mobilenet = MobileNetV2(include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [15]:
model = Sequential([
    mobilenet,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(30, activation='softmax')
]
)
model.compile(loss='categorical_crossentropy', metrics=['acc'])

In [16]:
batch_size = 10
steps_per_epoch = train_generator.n // batch_size

In [17]:
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, validation_data=val_generator) 
# steps_per_epoch = num_data // batch_size

Epoch 1/20
11/12 [==========================>...] - ETA: 15s - loss: 3.5564 - acc: 0.0455

KeyboardInterrupt: ignored

In [ ]:
pd.DataFrame(history.history).plot()

### Data Augmentation

Generating data on the fly, meaning that each epoch randomly given transformations will happen, number of images is kept

In [25]:
train_data = ImageDataGenerator(rescale=1/.255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='nearest',
                                preprocessing_function=preprocess_input)

val_data = ImageDataGenerator(rescale=1/.255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                fill_mode='nearest',
                                preprocessing_function=preprocess_input)


In [26]:
train_generator = train_data.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=10,
    seed=42,
)

val_generator = val_data.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=10,
    seed=42,
)

Found 120 images belonging to 30 classes.
Found 30 images belonging to 30 classes.


In [27]:
checkpoint = ModelCheckpoint('models/best.hdf5', monitor='val_loss', save_best_only=True, mode='auto')
callback_list = [checkpoint]
history = model.fit(train_generator, validation_data=val_generator, 
                    steps_per_epoch=steps_per_epoch, epochs=10, callbacks=callback_list)


Epoch 1/10
 3/12 [======>.......................] - ETA: 24s - loss: 2.9550 - acc: 0.2667

KeyboardInterrupt: ignored